# Emotion Detection - VGG9 CNN Model

### Data Preparation for training Emotion Dection model 
- Data Augmentation(rotation, zoom, flip etc) applied
- Rescale/ normalize the input image data

In [1]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os

num_classes = 7
img_rows, img_cols = 48, 48
batch_size = 16

train_data_dir = './train'
validation_data_dir = './validation'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      width_shift_range=0.4,
      height_shift_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

Using TensorFlow backend.


Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


## Building Keras CNN VGG9 Model (LittleVGG)

In [2]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense


def cnn_VGG9_model(activation="relu", kernel_initializer="he_normal"):

    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding = 'same', kernel_initializer=kernel_initializer,
                     input_shape = (img_rows, img_cols, 1)))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding = "same", kernel_initializer=kernel_initializer, 
                     input_shape = (img_rows, img_cols, 1)))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Block2: CONV => RELU => CONV => RELU => POOL
 
    model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Block3: CONV => RELU => CONV => RELU => POOL
 
    model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Block 4: CONV => RELU => CONV => RELU => POOL
    
    model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Block 5: FC => RELU layers
    model.add(Flatten())
    model.add(Dense(64, kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Block 6: FC => RELU layers
    model.add(Dense(64, kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Block 7: softmax classifier
    model.add(Dense(num_classes, kernel_initializer=kernel_initializer))
    model.add(Activation("softmax"))
    
    return model

## Model Training

In [3]:
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


def model_train(model, nb_train_samples, nb_validation_samples, epochs, CHECKPOINT_PATH):
    
    checkpoint = ModelCheckpoint(CHECKPOINT_PATH,
                                 monitor="val_loss",
                                 mode="min",
                                 save_best_only = True,
                                 verbose=1)

    earlystop = EarlyStopping(monitor = 'val_loss', 
                              min_delta = 0, 
                              patience = 3,
                              verbose = 1,
                              restore_best_weights = True)

    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

    # we put our call backs into a callback list
    callbacks = [earlystop, checkpoint, reduce_lr]

    # We use a very small learning rate 
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = Adam(lr=0.001),
                  metrics = ['accuracy'])


    history = model.fit_generator(
        train_generator,
        steps_per_epoch = nb_train_samples // batch_size,
        epochs = epochs,
        callbacks = callbacks,
        validation_data = validation_generator,
        validation_steps = nb_validation_samples // batch_size)
    
    return history

## Model Prediction

In [4]:
def model_predict(model, nb_train_samples, nb_validation_samples):

    # We need to recreate our validation generator with shuffle = false
    validation_generator = validation_datagen.flow_from_directory(
            validation_data_dir,
            color_mode = 'grayscale',
            target_size=(img_rows, img_cols),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False)

    class_labels = validation_generator.class_indices
    class_labels = {v: k for k, v in class_labels.items()}
    classes = list(class_labels.values())


    #Confution Matrix and Classification Report
    Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
    y_pred = np.argmax(Y_pred, axis=1)
    
    return y_pred, validation_generator

## Main Execution Block

In [48]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

CHECKPOINT_PATH = "./emotion_vgg9.h5"

epochs = 1
nb_train_samples = 28273
nb_validation_samples = 3589
view_model_parameters = False
view_confusion_matrix = False

model = cnn_VGG9_model(activation="elu", kernel_initializer="he_normal")

if view_model_parameters:
    print(model.summary())

model_train(model, nb_train_samples, nb_validation_samples, epochs, CHECKPOINT_PATH)

y_pred, validation_generator = model_predict(model, nb_train_samples, nb_validation_samples)


if view_confusion_matrix:
    
    print('Confusion Matrix & Classification Report : ')
    print(confusion_matrix(validation_generator.classes, y_pred))
    target_names = list(class_labels.values())
    print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

    plt.figure(figsize=(8,8))
    cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)

    plt.imshow(cnf_matrix, interpolation='nearest')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    _ = plt.xticks(tick_marks, classes, rotation=90)
    _ = plt.yticks(tick_marks, classes)

Epoch 1/1
1767/1767 [==============================] - 720s 408ms/step - loss: 2.0288 - accuracy: 0.2072 - val_loss: 1.8902 - val_accuracy: 0.2517

Epoch 00001: val_loss improved from inf to 1.89023, saving model to ./emotion_vgg9.h5
Found 3589 images belonging to 7 classes.


### Model validation on some random images

In [2]:
from keras.models import load_model
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing import image
import numpy as np
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import re

classifier = load_model('./emotion_vgg9.h5')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode = 'grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

Found 3589 images belonging to 7 classes.


In [50]:

def draw_test(name, pred, im, true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 160, 0, 0, 300 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, "predited - "+ pred, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.putText(expanded_image, "true - "+ true_label, (20, 120) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


def getRandomImage(path, img_width, img_height):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path, target_size = (img_width, img_height),grayscale=True), final_path, path_class

# dimensions of our images
img_width, img_height = 48, 48

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

files = []
predictions = []
true_labels = []

# predicting images
for i in range(0, 10):
    path = './validation/' 
    img, final_path, true_label = getRandomImage(path, img_width, img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x * 1./255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size = 10)
    predictions.append(classes)
    
for i in range(0, len(files)):
    image = cv2.imread((files[i]))
    image = cv2.resize(image, None, fx=3, fy=3, interpolation = cv2.INTER_CUBIC)
    draw_test("Prediction", class_labels[predictions[i][0]], image, true_labels[i])
    cv2.waitKey(0)

cv2.destroyAllWindows()

### Model Prediction : Realtime on our webcam
- Haarcascades model has been used for face detection in image

In [3]:
import cv2
import numpy as np
import time
from time import sleep
from keras.models import load_model
from keras.preprocessing.image import img_to_array

classifier = load_model('./emotion_vgg9.h5')

face_classifier = cv2.CascadeClassifier('./Haarcascades/haarcascade_frontalface_default.xml')

start_time = time.time()

def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation = cv2.INTER_AREA)
    except:
        return (x,w,y,h), np.zeros((48,48), np.uint8), img
    return (x,w,y,h), roi_gray, img

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    if np.sum([face]) != 0.0:
        roi = face.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        # make a prediction on the ROI, then lookup the class
        preds = classifier.predict(roi)[0]
        label = class_labels[preds.argmax()]  
        label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
        cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
    else:
        cv2.putText(image, "No Face Found", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
        
    cv2.imshow('All', image)
    if time.time() - start_time > 60 :
        break
        
cap.release()
cv2.destroyAllWindows()      

In [4]:
cv2.destroyAllWindows()